In [1]:
import numpy as np


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:

X = np.array(features)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)



In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]


In [11]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', max_iter=10, alpha=1e-4,
                    solver='adam', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

In [13]:
for Xtnn, ytnn, camId_t in zip(X_train, y_train, camId_train):
    X_train_pairs = []
    y_train_pair_lables = []
    for Xtnn2, ytnn2, camId_t2 in zip(X_train, y_train, camId_train):
        if ((camId_t == camId_t2) and (ytnn == ytnn2)):
            continue
        else:
            Xconcat = np.concatenate((Xtnn, Xtnn2), axis = None)
            X_train_pairs.append(Xconcat)
            if(ytnn == ytnn2):
                y_train_pair_lables.append(0)
            else:
                y_train_pair_lables.append(1)
    y_train_pair_lables = np.array(y_train_pair_lables)
    X_train_pairs = np.array(X_train_pairs)
    mlp.partial_fit(X_train_pairs, y_train_pair_lables)          

Iteration 2, loss = 4.65500333
Iteration 3, loss = 0.11085982
Iteration 4, loss = 0.01382720
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 5, loss = 0.00967025
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 6, loss = 0.00768398
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7, loss = 0.00623264
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 8, loss = 0.00513834
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 9, loss = 0.00429130
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 10, loss = 0.00362262
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 11, loss = 0.00334253
Iteration 12, loss = 0.00628797
Iteration 13, loss 

Iteration 75, loss = 0.23925267
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 76, loss = 0.78809900
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 77, loss = 0.18076617
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 78, loss = 0.15296884
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 79, loss = 0.92767538
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 80, loss = 2.08697163
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 81, loss = 0.23492790
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 82, loss = 0.16227949
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 83, lo

Iteration 142, loss = 0.14299786
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 143, loss = 0.13759081
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 144, loss = 0.13386086
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 145, loss = 0.12947740
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 146, loss = 0.12593259
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 147, loss = 0.12281183
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 148, loss = 0.17635965
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 149, loss = 0.12136917
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 209, loss = 0.15768245
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 210, loss = 0.14936349
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 211, loss = 0.14355125
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 212, loss = 0.16292517
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 213, loss = 0.14044068
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 214, loss = 0.13379428
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 215, loss = 0.12920924
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 216, loss = 0.12539206
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

Iteration 276, loss = 0.11558003
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 277, loss = 0.88569975
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 278, loss = 0.15210674
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 279, loss = 0.13120568
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 280, loss = 0.12619100
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 281, loss = 0.12248726
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 282, loss = 0.11948954
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 283, loss = 0.11688603
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteratio

...

Iteration 7089, loss = 0.03816677
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7090, loss = 0.03787414
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7091, loss = 0.03747727
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7092, loss = 0.03713189
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7093, loss = 0.03680367
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7094, loss = 0.03648399
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7095, loss = 0.03617577
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7096, loss = 0.03587268
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7155, loss = 0.03292512
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7156, loss = 0.03236824
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7157, loss = 0.03183929
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7158, loss = 0.03133182
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7159, loss = 0.03115194
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7160, loss = 0.03657217
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7161, loss = 0.03610511
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7162, loss = 0.03439710
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7221, loss = 0.02254039
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7222, loss = 0.10314152
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7223, loss = 0.02771903
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7224, loss = 0.02363587
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7225, loss = 0.02239700
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7226, loss = 0.02166893
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7227, loss = 0.02115844
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7228, loss = 0.02075871
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7287, loss = 0.00992430
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7288, loss = 0.00997701
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7289, loss = 0.01021242
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7290, loss = 0.00994851
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7291, loss = 0.01204282
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7292, loss = 0.01226154
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7293, loss = 0.01214456
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7294, loss = 0.01266295
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


Iteration 7353, loss = 0.01611195
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7354, loss = 0.01531480
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7355, loss = 0.01458108
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7356, loss = 0.01688718
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7357, loss = 0.01644607
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7358, loss = 0.01469541
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7359, loss = 0.01382805
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 7360, loss = 0.01354662
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


In [14]:
#y_train_pair_lables

In [15]:
#X_train_pairs

In [16]:
#X_train_pairs.shape

In [17]:
#from sklearn.neural_network import MLPRegressor
#mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='logistic', max_iter=10, alpha=1e-4,
#                    solver='adam', verbose=10, tol=1e-4, random_state=1,
#                    learning_rate_init=.1)

In [18]:
#mlp.fit(X_train_pairs, y_train_pair_lables)

In [19]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [20]:
from scipy.spatial import distance

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, metric = mlp, parameters = None):

    rank_accuracies = []
    AP = []

    # Break condition for testing
    #q = 0

    for query, camId_q, y_q in zip(X_query, camId_query, y_query):
        q_g_dists = []
        y_valid = []
        for gallery, camId_g, y_g  in zip(X_gallery, camId_gallery, y_gallery):
            if ((camId_q == camId_g) and (y_q == y_g)):
                continue
            else:
                if metric == mlp:
                    dist = mlp.predict(np.concatenate((query, gallery)).reshape((1,4096)))[0]
                elif metric == 'minkowski':
                    dist = distance.minkowski(query, gallery, parameters)
                else:
                    raise NameError('Specified metric not supported')           
                q_g_dists.append(dist)
                y_valid.append(y_g)
    
        tot_label_occur = y_valid.count(y_q)
    
        q_g_dists = np.array(q_g_dists)
        y_valid = np.array(y_valid)
    
        _indexes = np.argsort(q_g_dists)
    
        # Sorted distances and labels
        q_g_dists, y_valid = q_g_dists[_indexes], y_valid[_indexes]
    
        AP_, rank_A = get_acc_score(y_valid, y_q, tot_label_occur)
    
        AP.append(AP_)
        
        rank_accuracies.append(rank_A) 
    
        #if q  > 5:
        #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))
    
    return rank_accuracies, mAP

In [22]:
rank_accuracies, mAP = evaluate_metric(X_query, camId_query, y_query,
                                       X_gallery, camId_gallery, y_gallery,
                                       metric = mlp,
                                       parameters = None)

Accuracies by Rank:
Rank  1  = 0.07% 	 Rank  2  = 0.07% 	 Rank  3  = 0.14% 	 Rank  4  = 0.29% 	 Rank  5  = 0.43%
Rank  6  = 0.43% 	 Rank  7  = 0.50% 	 Rank  8  = 0.50% 	 Rank  9  = 0.50% 	 Rank  10  = 0.50%
Rank  11  = 0.64% 	 Rank  12  = 0.71% 	 Rank  13  = 0.71% 	 Rank  14  = 0.71% 	 Rank  15  = 0.71%
Rank  16  = 0.71% 	 Rank  17  = 0.71% 	 Rank  18  = 0.79% 	 Rank  19  = 0.79% 	 Rank  20  = 0.93%
Rank  21  = 0.93% 	 Rank  22  = 0.93% 	 Rank  23  = 0.93% 	 Rank  24  = 0.93% 	 Rank  25  = 1.00%
Rank  26  = 1.07% 	 Rank  27  = 1.21% 	 Rank  28  = 1.36% 	 Rank  29  = 1.36% 	 Rank  30  = 1.43%
mAP = 0.35%
